In [67]:
import pandas as pd

raw = [
    {"date":"2026-01-01", "time":"09:10", "store":"A", "menu":"Americano", "price":"4,500원", "qty":"2", "paid":"TRUE"},
    {"date":"2026/01/01", "time":"09:12", "store":"A", "menu":"Latte",     "price":"5000",   "qty":1,   "paid":"True"}, # /
    {"date":"2026-01-02", "time":"12:30", "store":"A", "menu":"Latte",     "price":None,     "qty":2,   "paid":"FALSE"},
    {"date":"2026-01-03", "time":"18:05", "store":"B", "menu":"Mocha",     "price":"5500",   "qty":None,"paid":True},
    {"date":"2026-01-03", "time":"18:05", "store":"B", "menu":"Mocha",     "price":"5500",   "qty":None,"paid":True},  # 중복
    {"date":"2026-01-04", "time":"08:55", "store":"B", "menu":"Americano ", "price":"4500",  "qty":"1", "paid":"TRUE"}, # 공백
    {"date":"2026-01-04", "time":"08:58", "store":"A", "menu":"latte",     "price":"5,000",  "qty":"3", "paid":"TRUE"}, # 소문자
]
df = pd.DataFrame(raw)

df

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE
3,2026-01-03,18:05,B,Mocha,5500,None,True
4,2026-01-03,18:05,B,Mocha,5500,None,True
5,2026-01-04,08:55,B,Americano,4500,1,TRUE
6,2026-01-04,08:58,A,latte,"5,000",3,TRUE


In [68]:
df.shape

(7, 7)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    7 non-null      object
 1   time    7 non-null      object
 2   store   7 non-null      object
 3   menu    7 non-null      object
 4   price   6 non-null      object
 5   qty     5 non-null      object
 6   paid    7 non-null      object
dtypes: object(7)
memory usage: 524.0+ bytes


In [70]:
df.describe(include="all")

,date,time,store,menu,price,qty,paid
count,7,7,7,7,6,5,7
unique,5,6,2,5,5,5,4
top,2026-01-04,18:05,A,Latte,5500,2,TRUE
freq,2,2,4,2,2,1,3


- count: null 제외 관측수
- unique: 중복이 제거된 개체 수
- top: 최빈값
- frequency: 최빈값의 수

## 3) 인덱싱 핵심: .loc vs .iloc (여기서 실수하면 모든 게 꼬임)

### 개념(딱 2줄)

- .loc = 라벨(feature) 기반 (행/열 이름으로 접근)

- .iloc = 위치(순서) 기반 (0번째, 1번째…로 접근)

### 실습 1: “한 행” 뽑기
```python
df.loc[0]      # 0번 인덱스 라벨
df.iloc[0]     # 0번째 위치
```

In [71]:
df.loc[0]

date     2026-01-01
time          09:10
store             A
menu      Americano
price        4,500원
qty               2
paid           TRUE
Name: 0, dtype: object

In [72]:
df.iloc[0]

date     2026-01-01
time          09:10
store             A
menu      Americano
price        4,500원
qty               2
paid           TRUE
Name: 0, dtype: object

실습 2: “행 범위” 슬라이싱 실수 포인트

```python
df.loc[0:2]    # 끝 포함(텍스트 기반이기 떄문)
df.iloc[0:2]   # 끝 미포함
```

In [73]:
df.loc[0:2]

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE


In [74]:
df.iloc[0:2]

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True


### 4) 열 선택 실전: 필요한 컬럼만 남기기

오늘 회의용 테이블은 이 컬럼만 필요합니다:

- date, time, store, menu, price, qty, paid

In [75]:
cols = ["date","time","store","menu","price","qty","paid"]
df2 = df[cols].copy()
df2.head() # 상위 5개

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,TRUE
1,2026/01/01,09:12,A,Latte,5000,1,True
2,2026-01-02,12:30,A,Latte,None,2,FALSE
3,2026-01-03,18:05,B,Mocha,5500,None,True
4,2026-01-03,18:05,B,Mocha,5500,None,True


### 5) 조건 필터링(불리언 인덱싱): “결제 완료 + A매장 + 오전”

- 규칙 1) & | ~ 는 괄호 필수

- 규칙 2) 실무 패턴은 이거 하나: df.loc[조건, 컬럼]

- Step 1) 먼저 paid 값을 통일(소문자/대문자/True 섞임)

In [76]:
df2["paid"] = df2["paid"].astype(str).str.strip().str.lower()
df2["paid"].value_counts()

paid
true     6
false    1
Name: count, dtype: int64

- Step 2) 결제 완료만 추출

In [77]:
# 1. cond_paid라는 isin() 조건에 따라 [True, False, True, ...]값을 가진 Series 생성
cond_paid = df2["paid"].isin(["true"])
# 2. 그 시리즈를 변수로 불리언 인덱싱 하여 새로운 데이터프레임(서브셋) 생성
    # loc의 3가지 모드
    # 이름(Label) 모드: "행의 이름"이 들어오면 → 해당 행을 찾습니다.
    # 슬라이싱(Slicing) 모드: start:end가 들어오면 → 범위를 자릅니다.
    # 불리언(Boolean) 모드: True/False 리스트가 들어오면 → 필터링(Masking)을 수행합니다.
paid_df = df2.loc[cond_paid]
# 3. paid_df 출력
paid_df

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,true
1,2026/01/01,09:12,A,Latte,5000,1,true
3,2026-01-03,18:05,B,Mocha,5500,None,true
4,2026-01-03,18:05,B,Mocha,5500,None,true
5,2026-01-04,08:55,B,Americano,4500,1,true
6,2026-01-04,08:58,A,latte,"5,000",3,true


- Step 3) A매장 + 오전(09시대)만 더 좁히기

In [78]:
# 1. store==A인 값많 True
cond_storeA = (df2["store"] == "A")
# 2. 09로 시작하는 값만 True
cond_morning = df2["time"].str.startswith("09")

# 3. cond_paid까지 전부 True인 row만 인덱싱
df_morning_A = df2.loc[cond_paid & cond_storeA & cond_morning, ["date","time","store","menu","price","qty"]]
# 4. 출력
df_morning_A

,date,time,store,menu,price,qty
0,2026-01-01,09:10,A,Americano,"4,500원",2
1,2026/01/01,09:12,A,Latte,5000,1


## 6) 정렬: “메뉴별 판매수량 TOP”을 뽑을 준비

정렬은 2가지가 기본입니다.

- sort_values = 값 기준 정렬

- sort_index = 인덱스 기준 정렬

In [79]:
# 정렬 by date, time / 오름차순? [True!, True!] / 5개만!
df2.sort_values(by=["date","time"], ascending=[True, True]).head()

,date,time,store,menu,price,qty,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,true
2,2026-01-02,12:30,A,Latte,None,2,false
3,2026-01-03,18:05,B,Mocha,5500,None,true
4,2026-01-03,18:05,B,Mocha,5500,None,true
5,2026-01-04,08:55,B,Americano,4500,1,true


## 7) 클리닝 기본기 4종 세트 (오늘의 하이라이트)

### 7-1) rename / drop: 컬럼명 정리(선택)

- 실무에서는 컬럼명 통일이 엄청 중요합니다.

-  오늘은 간단히 넘어가도 OK.

In [80]:
# 컬럼명 재할당 {기존값 : 새로운값}
df2 = df2.rename(columns={"qty":"quantity"})
df2

,date,time,store,menu,price,quantity,paid
0,2026-01-01,09:10,A,Americano,"4,500원",2,true
1,2026/01/01,09:12,A,Latte,5000,1,true
2,2026-01-02,12:30,A,Latte,None,2,false
3,2026-01-03,18:05,B,Mocha,5500,None,true
4,2026-01-03,18:05,B,Mocha,5500,None,true
5,2026-01-04,08:55,B,Americano,4500,1,true
6,2026-01-04,08:58,A,latte,"5,000",3,true


### 7-2) 문자열 정리: menu 공백/대소문자 통일

In [81]:
df2["menu"] = df2["menu"].astype(str).str.strip().str.title() # 맨 앞글자만 대문자
df2["menu"].value_counts() # 각 변수 freq 확인

menu
Latte        3
Americano    2
Mocha        2
Name: count, dtype: int64

### 7-3) price 정리: “원”, “,” 제거하고 숫자로 만들기

In [84]:
df2["price"] = (
    df2["price"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
    .str.strip()
)

df2["price"] = pd.to_numeric(df2["price"], errors="coerce")
df2["price"].isnull().sum()

np.int64(1)

In [85]:
df2["price"] = (
    df2["price"]
    .astype(str)
    .str.replace(r"[^0-9]", "", regex=True)  # 숫자(0-9) 제외 전부 제거
)

df2["price"] = pd.to_numeric(df2["price"], errors="coerce")
df2["price"].isnull().sum()

np.int64(1)

### 7-4) 결측치 처리: dropna vs fillna (전략 선택)

가격(price)이 비어 있으면 매출 계산이 안 됩니다.

실무에서는 보통 2가지 선택:

- 삭제(drop): 데이터가 적어도 괜찮고, 결측이 중요 변수면 제거

- 대체(fill): 평균/중앙값/메뉴별 평균 등으로 대체

오늘은 초보자 기준으로 “간단 전략”으로 갑니다.

In [88]:
# quantity도 숫자로 정리
df2["quantity"] = pd.to_numeric(df2["quantity"], errors="coerce")

# 핵심 변수 결측은 삭제(간단 버전)
clean = df2.dropna(subset=["price","quantity"]).copy() # null 포함 row 모두 삭제
clean.shape # 삭제 row 제외 4개 row 남음

(4, 7)

In [92]:
clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 0 to 6
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      4 non-null      object 
 1   time      4 non-null      object 
 2   store     4 non-null      object 
 3   menu      4 non-null      object 
 4   price     4 non-null      float64
 5   quantity  4 non-null      float64
 6   paid      4 non-null      object 
dtypes: float64(2), object(5)
memory usage: 256.0+ bytes


## 8) 중복 처리: duplicated / drop_duplicates

In [95]:
clean.duplicated().sum()


np.int64(0)

In [96]:
clean = clean.drop_duplicates(keep="first") # 중복값은 앞 값만 남기고 drop
clean.duplicated().sum()

np.int64(0)

## 9) 미니 결과물 만들기: “매출 컬럼 추가 + 회의용 테이블 정리”

In [101]:
clean["date"] = pd.to_datetime(df["date"], errors='coerce', format='mixed')
clean["date"]

0   2026-01-01
1   2026-01-01
5   2026-01-04
6   2026-01-04
Name: date, dtype: datetime64[ns]

In [102]:
clean["sales"] = clean["price"] * clean["quantity"]

meeting_table = clean.loc[:, ["date","time","store","menu","price","quantity","sales"]].sort_values(
    by=["date","store","time"],
    ascending=[True, True, True]
)

meeting_table

,date,time,store,menu,price,quantity,sales
0,2026-01-01,09:10,A,Americano,45000.0,2.0,90000.0
1,2026-01-01,09:12,A,Latte,50000.0,1.0,50000.0
6,2026-01-04,08:58,A,Latte,50000.0,3.0,150000.0
5,2026-01-04,08:55,B,Americano,45000.0,1.0,45000.0


## 10) 저장: 2회차 결과물을 파일로 남기기 (3회차를 위해 필수)

In [104]:
meeting_table.to_csv("cafe_sales_clean_v1.csv", index=False, encoding="utf-8-sig")

In [108]:
df_meeting=pd.read_csv("cafe_sales_clean_v1.csv")
df_meeting

,date,time,store,menu,price,quantity,sales
0,2026-01-01,09:10,A,Americano,45000.0,2.0,90000.0
1,2026-01-01,09:12,A,Latte,50000.0,1.0,50000.0
2,2026-01-04,08:58,A,Latte,50000.0,3.0,150000.0
3,2026-01-04,08:55,B,Americano,45000.0,1.0,45000.0
